# Импортируем библиотеки

In [107]:
import pandas as pd
import numpy as np

# Вспомогательные функции расчетов

In [119]:
def parse_segment(segment):
    start, end = map(int, segment.split("-"))
    return start, end


def iou(segment_q, segment_t):
    start_q, stop_q = parse_segment(segment_q)
    start_t, stop_t = parse_segment(segment_t)
    
    intersection_start = max(start_q, start_t)
    intersection_end = min(stop_q, stop_t)

    intersection_length = max(0, intersection_end - intersection_start)
    union_length = (stop_q - start_q) + (stop_t - start_t) - intersection_length

    iou = intersection_length / union_length if union_length > 0 else 0
    return iou


def f1(tp, fp, fn):
    precision = tp / (tp + fp + 1e-6)
    recall = tp / (tp + fn + 1e-6)
    
    print(f'Precision = {precision}')
    print(f'Recall = {recall}')
    
    return 2 * (precision * recall) / (precision + recall + 1e-6)

def final_metric(tp, fp, fn, final_iou):
    f = f1(tp, fp, fn)
    
    print(f'IOU = {final_iou}')
    
    return 2 * (final_iou * f) / (final_iou + f + 1e-6)


# Подгружаем свои данные

In [108]:
orig, target = pd.read_csv('../orig_df.csv'), pd.read_csv('../target_df.csv')

In [109]:
orig

,ID-piracy,SEG-piracy,ID-license,SEG-license
0,id-1,3-21,id-3,10-30
1,id-1,45-60,id-8,150-165
2,id-2,15-35,id-5,10-30
3,id-4,145-300,id-8,1040-1195
4,id-4,0-19,id-8,1040-1195


In [110]:
orig_dict = orig.groupby(['ID-piracy', 'ID-license']).count().to_dict()['SEG-piracy']
orig_dict

{('id-1', 'id-3'): 1,
 ('id-1', 'id-8'): 1,
 ('id-2', 'id-5'): 1,
 ('id-4', 'id-8'): 2}

In [111]:
target

,ID-piracy,SEG-piracy,ID-license,SEG-license
0,id-1,0-25,id-3,10-35
1,id-1,12-25,id-3,100-115
2,id-2,0-12,id-8,132-144
3,id-7,85-110,id-10,30-65
4,id-4,120-250,id-8,1000-1130
5,id-1,120-250,id-10,1000-1130


In [112]:
target_dict = target.groupby(['ID-piracy', 'ID-license']).count().to_dict()['SEG-piracy']
target_dict

{('id-1', 'id-10'): 1,
 ('id-1', 'id-3'): 2,
 ('id-2', 'id-8'): 1,
 ('id-4', 'id-8'): 1,
 ('id-7', 'id-10'): 1}

# Подсчет FP, TP, FN

In [113]:
fn = 0
fp = 0
tp = 0

for ids, count in orig_dict.items():
    if ids not in target_dict:
        fn += count # модель не нашла что то из оригинальной таблицы
    elif target_dict[ids] > count:
        fp += target_dict[ids] - count # модель нашла больше совпадений чем в оригинальной таблице
        tp += min(target_dict[ids], count) # тогда для истинных совпадений совпадений берем наименьшее количество
    elif target_dict[ids] < count:
        fn += count - target_dict[ids] # модель нашла меньше совпадений чем в оригинальной таблице
        tp += min(target_dict[ids], count) # тогда для истинных совпадений совпадений берем наименьшее количество

for ids, count in target_dict.items():
    if ids not in orig_dict:
        fp += count # модель нашла то, чего не было в оригинальной таблице

print(fp, fn, tp)

4 3 2


# Подсчет IOU

In [97]:
ious = []

# Подсчет IOU для каждой отдельной строки из orig
for i, row in orig.iterrows():
    max_iou = 0
    merged = pd.merge(
        row.to_frame().T,
        target,
        'left',
        left_on=['ID-piracy', 'ID-license'],
        right_on = ['ID-piracy', 'ID-license']
    ).dropna()
    
    # Выбор наилучшего IOU по всем совпадениям из target
    if len(merged) > 0:
        for j, row1 in merged.iterrows():
            final_iou = iou(row1['SEG-piracy_x'], row1['SEG-piracy_y']) * iou(row1['SEG-license_x'], row1['SEG-license_y'])
            if final_iou > max_iou:
                max_iou = final_iou
    
    ious.append(max_iou)

In [98]:
ious

[0.576, 0, 0, 0.2692307692307693, 0]

In [115]:
print(f'F1 = {f1(tp, fp, fn)}')

Precision = 0.33333327777778704
Recall = 0.399999920000016
F1 = 0.3636358016535793


In [116]:
final_iou = sum(ious) / (len(ious) + fp) # чтобы учесть количество лишних в IOU добавим в знаменатель их количество (так как их IOU = 0)
print(f'IOU = {final_iou}')

IOU = 0.09391452991452992


In [121]:
print(f'Metric = {final_metric(tp, fp, fn, final_iou)}')

Precision = 0.33333327777778704
Recall = 0.399999920000016
IOU = 0.09391452991452992
Metric = 0.1492758648755265
